# Summarization

## Getting started


- Get your [API Key](https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key) from OpenAI.com.

- Save your API key in `.env` file as `OPENAI_API_KEY = 'sk-...XXXX'`

- Load it with [python-dotenv](https://pypi.org/project/python-dotenv/) 

- or save it in `.streamlit/secrets.toml` and load it with [tomli](https://pypi.org/project/tomli/).


Resources: 
- https://github.com/gkamradt/langchain-tutorials/blob/main/getting_started/Quickstart%20Guide.ipynb
- https://www.youtube.com/watch?v=kYRB-vJFy38&list=PLqZXAkvF1bPNQER9mLmDbntNfSpzdDIU5&index=2
- https://python.langchain.com/docs/use_cases/summarization

In [ ]:
# from dotenv import load_dotenv

# load_dotenv()  # take environment variables from .env.

# Code of your application, which uses environment variables (e.g. from `os.environ` or
# `os.getenv`) as if they came from the actual environment

In [1]:
import tomli, os
with open("../.streamlit/secrets.toml","rb") as f:
    secrets = tomli.load(f)
os.environ["OPENAI_API_KEY"] = secrets["OPENAI_API_KEY"]

In [ ]:
# # Check that the OpenAI API key is correctly loaded as env variable
import os
os.environ['OPENAI_API_KEY']

## LangChain

In [19]:
from langchain.llms import OpenAI
llm = OpenAI(
    # api_key=os.environ['OPENAI_API_KEY'],
)
joke = llm('tell me a joke')
print(joke)



Q: Why don't scientists trust atoms?
A: Because they make up everything!


In [20]:
llm.model_name

'text-davinci-003'

Try out your first call to OpenAI model `gpt-3.5-turbo` that powers ChatGPT. 

The cost of the API is: 
- input: $0.001 per 1k tokens
- output: $0.002 per 1k tokens

[(November 2023 updated prices)](https://openai.com/pricing#gpt-3-5-turbo)

In [21]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
chat = ChatOpenAI()
text = "Tell me a joke"
messages = [HumanMessage(content=text)]
res = chat.invoke(messages)
print(res.content)

Sure, here's a classic one for you:

Why don't scientists trust atoms?

Because they make up everything!


## Stuff

In [ ]:
import os, webvtt
files = os.listdir('../data/vtt')
print(files) # ['captions.vtt', 'sample.vtt']
file = files[0]

In [34]:
caption = webvtt.read('../data/vtt/'+file)
for cap in caption[1:5]:
    # print(f'From {caption.start} to {caption.end}')
    # print(caption.raw_text)
    print(cap.text)

I want to start doing this experiment where.
We have a conversation we record it's generating a VTT file.
And I have a parser. I developed a small app in Python that can retrieve the VTT file process it.
And then.


We can extract the text from the conversation and save it as a plain text file.

In [64]:
txt = file.replace('.vtt','.txt')
m = [cap.raw_text for cap in caption]
sep = '\n'
convo = sep.join(m)
with open('../data/txt/'+txt,mode='w') as f:
    f.write(convo)

Let's count the numbers of token in the conversation.

In [36]:
import tiktoken
encoding_name = 'cl100k_base'
encoding = tiktoken.get_encoding(encoding_name)
num_tokens = len(encoding.encode(convo))
num_tokens

150

In [7]:
from langchain.chains.summarize import load_summarize_chain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader, TextLoader

In [37]:
loader = TextLoader('../data/txt/'+txt)
docs = loader.load()
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
chain = load_summarize_chain(llm, chain_type="stuff")
chain.run(docs)

'Yann wants to improve his French accent and plans to conduct an experiment where he records conversations and generates VTT files. He has developed a Python app to process the VTT files and wants to use the ChatGPT API to further analyze them. Mike has not yet used the API due to lack of time.'

In [49]:
len(docs[0].page_content) # number of characters in the document

550

Try summarizing a longer text, like the content of the LangChain doc page.

In [78]:
def summarize(page, model = "gpt-3.5-turbo"):  
    loader = WebBaseLoader(page)
    docs = loader.load()
    llm = ChatOpenAI(temperature=0, model_name=model)
    chain = load_summarize_chain(llm, chain_type="stuff")
    return chain.run(docs)

In [83]:
page = "https://python.langchain.com/docs/use_cases/summarization"
try:
    summary = summarize(page)
    print(summary)
except Exception as e:
    print(str(e))

Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens. However, your messages resulted in 7705 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}


In [6]:
page = "https://python.langchain.com/docs/use_cases/summarization"
summarize(page,model = "gpt-3.5-turbo-16k")

'The LangChain platform offers tools for document summarization using large language models (LLMs). There are three approaches to document summarization: "stuff," "map-reduce," and "refine." The "stuff" approach involves inserting all documents into a single prompt, while the "map-reduce" approach summarizes each document individually and then combines the summaries into a final summary. The "refine" approach iteratively updates the summary by passing each document and the current summary through an LLM chain. The platform provides pre-built chains for each approach, and users can customize prompts and LLM models. Additionally, the platform offers the option to split long documents into chunks and summarize them in a single chain.'

## Map Reduce

Youtube transcript

https://www.geeksforgeeks.org/python-downloading-captions-from-youtube/

### V1

In [53]:
from youtube_transcript_api import YouTubeTranscriptApi
video_id = "f9_BWhCI4Zo"
srt = YouTubeTranscriptApi.get_transcript(video_id)
srt

[{'text': 'so you got yourself in a bit of trouble',
  'start': 0.06,
  'duration': 3.96},
 {'text': 'because open AI returns an error to you',
  'start': 1.8,
  'duration': 4.62},
 {'text': 'and that error says that you have', 'start': 4.02, 'duration': 5.7},
 {'text': "exceeded the token length that's an",
  'start': 6.42,
  'duration': 4.62},
 {'text': "issue and we're going to show you four",
  'start': 9.72,
  'duration': 2.999},
 {'text': 'different ways on how to fix that issue',
  'start': 11.04,
  'duration': 3.42},
 {'text': "so first let's set up the problem here",
  'start': 12.719,
  'duration': 3.721},
 {'text': "just one more time I'm going to copy and",
  'start': 14.46,
  'duration': 4.62},
 {'text': 'paste a short passage into the', 'start': 16.44, 'duration': 4.2},
 {'text': 'playground on open AI this is the same',
  'start': 19.08,
  'duration': 3.06},
 {'text': "for the API and I'm going to say hey",
  'start': 20.64,
  'duration': 3.78},
 {'text': 'please summari

In [72]:
import tiktoken
def num_tokens(string: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding_name = 'cl100k_base'
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [77]:
sep = '\n'
caption = sep.join([s['text'] for s in srt])
num_tokens(caption)

4150

In [71]:
# Save srt as a webvtt file
import webvtt
from datetime import datetime

# Create a new WebVTT file
vtt = webvtt.WebVTT()

# Convert each subtitle in the srt to a WebVTT caption
for subtitle in srt:
    start = datetime.fromtimestamp(subtitle['start']).strftime('%H:%M:%S.%f')[:-3]
    end = datetime.fromtimestamp(subtitle['start'] + subtitle['duration']).strftime('%H:%M:%S.%f')[:-3]
    text = subtitle['text']
    caption = webvtt.Caption(start, end, text)
    vtt.captions.append(caption)

# Save the WebVTT file
vtt.save('../data/vtt/subtitles.vtt')


### V2

In [87]:
from youtube_transcript_api.formatters import WebVTTFormatter
from youtube_transcript_api import YouTubeTranscriptApi
video_id = "2xxziIWmaSA" # https://www.youtube.com/watch?v=2xxziIWmaSA
transcript = YouTubeTranscriptApi.get_transcript(video_id)
formatter = WebVTTFormatter()
formatted_captions = formatter.format_transcript(transcript)
vtt_file = f'../data/vtt/subtitles-{video_id}.vtt'
with open(vtt_file, 'w') as f:
    f.write(formatted_captions)

In [85]:
sep = '\n'
caption = sep.join([s['text'] for s in transcript])
num_tokens(caption)

10336

In [100]:
# Turn VTT file into TXT file
txt_file = vtt_file.replace('vtt','txt')
with open(txt_file,mode='w') as f:
    f.write(caption)

In [92]:
from langchain.document_loaders import TextLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI

In [101]:
loader = TextLoader(txt_file)
doc = loader.load()

In [109]:
def doc_summary(docs):
    print (f'You have {len(docs)} document(s)')
    num_words = sum([len(doc.page_content.split(' ')) for doc in docs])
    print (f'You have roughly {num_words} words in your docs')
    print ()
    print (f'Preview: \n{docs[0].page_content.split(". ")[0][0:42]}')

In [110]:
doc_summary(doc)

You have 1 document(s)
You have roughly 7443 words in your docs

Preview: 
hello good people have you ever wondered
w


In [111]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 0,
    length_function = num_tokens,
)
docs = text_splitter.split_documents(doc)
doc_summary(docs)

You have 11 document(s)
You have roughly 7453 words in your docs

Preview: 
hello good people have you ever wondered
w


In [112]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo')
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
chain.run(docs)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"hello good people have you ever wondered
what Lang chain was or maybe you've
heard about it and you've played around
with a few sections but you're not quite
sure where to look next well in this
video we're going to be covering all of
the lane chain Basics with the goal of
getting you building and having fun as
quick as possible my name is Greg and
I've been having a ton of fun building
out apps in langchain now I share most
of my work on Twitter so if you want to
go check it out links in the description
you can go follow along with me now this
video is going to be based off of the
new conceptual docs from lanechain and
the reason why I'm doing a video here is
because it takes all the technical
pieces and abstracts them up into more
theoretical qualitative aspects of Lane
chain which I think is extremely helpful
for it and in order t

"The video introduces Lang chain, a framework for developing applications powered by language models. It explains the components and benefits of Lang chain, and mentions a companion cookbook for further examples. The video discusses different types of models and how they interact with text, including language models, chat models, and text embedding models. It explains the use of prompts, prompt templates, and example selectors. The process of importing and using a semantic similarity example selector is described. The video also discusses the use of text splitters, document loaders, and retrievers. The concept of vector stores and various platforms are mentioned. The video explains how chat history can improve language models and introduces the concept of chains. It demonstrates the creation of different chain types in Lang chain, such as location, meal, and summarization chains. The concept of agents and their application in decision making is discussed, along with the process of crea

## Refine

In [113]:
chain = load_summarize_chain(llm, chain_type="refine", verbose=True)
chain.run(docs)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"hello good people have you ever wondered
what Lang chain was or maybe you've
heard about it and you've played around
with a few sections but you're not quite
sure where to look next well in this
video we're going to be covering all of
the lane chain Basics with the goal of
getting you building and having fun as
quick as possible my name is Greg and
I've been having a ton of fun building
out apps in langchain now I share most
of my work on Twitter so if you want to
go check it out links in the description
you can go follow along with me now this
video is going to be based off of the
new conceptual docs from lanechain and
the reason why I'm doing a video here is
because it takes all the technical
pieces and abstracts them up into more
theoretical qualitative aspects of Lane
chain which I think is extremely helpful
for it and in order to u

'The video explores the concept of semantic similarity example selection using language models and emphasizes the importance of output parsers. It introduces the Lang chain framework for automating prompt formatting and parsing language model outputs. The video also discusses the use of output parsing and document loaders within the Lang chain framework. Additionally, it highlights the significance of text splitting or chunking for inputting smaller pieces of text into language models. The video introduces the concept of retrievers, specifically the vector store retriever, for combining documents with language models. It also mentions the significance of vector stores and provides examples of popular ones. The new context discusses the use of embeddings and memory in language models, as well as the functionality of chains in combining different language model calls and actions automatically. The video demonstrates the use of the Lang chain framework for creating classic dishes based on

I'll implement my own text splitter and summarizer using the refine method.

In [115]:
def my_text_splitter(text,chunk_size=3000):
    # Split text into chunks based on space or newline
    chunks = text.split()

    # Initialize variables
    result = []
    current_chunk = ""

    # Concatenate chunks until the total length is less than 4096 tokens
    for chunk in chunks:
        # if len(current_chunk) + len(chunk) < 4096:
        if num_tokens(current_chunk+chunk) < chunk_size:
            current_chunk += " " + chunk if current_chunk else chunk
        else:
            result.append(current_chunk.strip())
            current_chunk = chunk
    if current_chunk:
        result.append(current_chunk.strip())

    return result

In [122]:
chunks = my_text_splitter(caption)
chunks

["hello good people have you ever wondered what Lang chain was or maybe you've heard about it and you've played around with a few sections but you're not quite sure where to look next well in this video we're going to be covering all of the lane chain Basics with the goal of getting you building and having fun as quick as possible my name is Greg and I've been having a ton of fun building out apps in langchain now I share most of my work on Twitter so if you want to go check it out links in the description you can go follow along with me now this video is going to be based off of the new conceptual docs from lanechain and the reason why I'm doing a video here is because it takes all the technical pieces and abstracts them up into more theoretical qualitative aspects of Lane chain which I think is extremely helpful for it and in order to understand this a little bit better I've created a companion for this video and that is the Lang chain cookbook links in the description if you want to

In [127]:
import openai
def summarize(text, context = 'summarize the following text:', model = 'gpt-3.5-turbo'):
    """Returns the summary of a text."""
    completion = openai.chat.completions.create(
        model = model,
        messages=[
        {'role': 'system','content': context},
        {'role': 'user', 'content': text}
            ]
    )
    return completion.choices[0].message.content

In [128]:
def refine(summary, chunk,  model = 'gpt-3.5-turbo'):
    """Refine the summary with each new chunk of text"""
    context = "Refine the summary with the following context: " + summary
    summary = summarize(chunk, context, model)
    return summary

In [131]:
# Requires initialization with summary of first chunk 
summary = summarize(chunks[0])
for chunk in chunks[1:]:
    summary = refine(summary, chunk)
summary

'The video provides an introduction to Lang chain, a framework for building applications powered by language models. It explains the various components of Lang chain and highlights its benefits. The author also provides a companion document and code samples for further exploration. The video concludes with an overview of chains and agents, which allow for combining different language model calls and enable decision-making. The author encourages viewers to subscribe and stay tuned for part two, where they will explore actual use cases. They also invite comments and questions.'